In [ ]:
import pandas as pd
import re
import os
import zipfile
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import seaborn as sns



: 

In [ ]:

# Carregar o arquivo CSV
IPTU_util = pd.read_csv("IPTU_2024/IPTU_2024.csv", sep=";", encoding="latin1", quotechar='"')

# Selecionar e renomear as colunas
IPTU_util = IPTU_util.rename(columns={
    "NUMERO.DO.CONTRIBUINTE": "sql",
    "NUMERO.DO.CONDOMINIO": "condominio",
    "NOME.DE.LOGRADOURO.DO.IMOVEL": "rua",
    "NUMERO.DO.IMOVEL": "numero",
    "AREA.DO.TERRENO": "area_terreno",
    "BAIRRO.DO.IMOVEL": "bairro",
    "CEP.DO.IMOVEL": "CEP",
    "VALOR.DO.M2.DO.TERRENO": "valor_m2_terreno",
    "AREA.CONSTRUIDA": "area_construida",
    "VALOR.DO.M2.DE.CONSTRUCAO": "valor_m2_construcao",
    "AREA.OCUPADA": "area_ocupada",
    "TIPO.DE.PADRAO.DA.CONSTRUCAO": "tipo"
})

# Criar as colunas setor, quadra e lote
IPTU_util["setor"] = IPTU_util["sql"].astype(str).str[:3]
IPTU_util["quadra"] = IPTU_util["sql"].astype(str).str[3:6]

# Definir lote, verificando se é um condomínio
IPTU_util["lote"] = IPTU_util.apply(
    lambda row: row["sql"][6:10] if row["condominio"] == "00-0" else f"CD{row['condominio'][:2]}", axis=1
)

# Definir a coluna residencial com base no tipo
IPTU_util["residencial"] = IPTU_util["tipo"].str.contains("Residencial", case=False, na=False)

# Exibir as primeiras linhas do dataframe
print(IPTU_util.head())


In [ ]:
# Filtrar apenas imóveis residenciais e agregar os dados
IPTU = (IPTU_util[IPTU_util["residencial"]]
    .groupby(["setor", "quadra", "lote"], as_index=False)
    .agg(
        unidades=("sql", "count"),
        area_terreno=("area_terreno", "median"),
        area_construida=("area_construida", "sum"),
        valor_terreno=("valor_m2_terreno", "median"),
        valor_construcao=("valor_m2_construcao", "median"),
        residencial=("residencial", "median")
    )
)

In [ ]:
if "zip" not in os.listdir("dados/lotes"):
    for file in os.listdir("dados/lotes/zip"):
        file_name = re.sub(r"\.zip", "", file)
        with zipfile.ZipFile(f"dados/lotes/zip/{file}", 'r') as zip_ref:
            for ext in ["shp", "dbf", "shx"]:
                try:
                    zip_ref.extract(f"{file_name}/{file_name}.{ext}", path="dados/lotes/unzip")
                except KeyError:
                    print(f"Arquivo {file_name}.{ext} não encontrado no zip.")

# Criar diretório 'combinados' se não existir
if not os.path.exists("dados/lotes/combinados"):
    os.makedirs("dados/lotes/combinados", exist_ok=True)

In [ ]:
lotes_combinados = [
    gpd.read_file(f"dados/lotes/unzip/{file}/{file}.shp")
    for file in os.listdir("dados/lotes/unzip")
]

lotes_combinados = gpd.GeoDataFrame(pd.concat(lotes_combinados, ignore_index=True))
lotes_combinados.set_crs(epsg=31983, inplace=True)

# Salvar o arquivo combinado
lotes_combinados.to_file("dados/lotes/combinados/lotes_combinados.shp")


In [ ]:
# Carregar o shapefile
censo = gpd.read_file("SP_Malha_Preliminar_2022/SP_Malha_Preliminar_2022.shp")

# Filtrar apenas o município específico
censo = censo[censo["CD_MUN"] == "3550308"]

# Transformar o sistema de coordenadas para EPSG:31983
censo = censo.to_crs(epsg=31983)

# Selecionar a coluna desejada e calcular a área do setor censitário
censo = censo[["CD_SETOR"]].rename(columns={"CD_SETOR": "id_setor_censitario"})
censo["area_setor"] = censo.geometry.area.astype(float)

# Converter a geometria para POLYGON
censo = censo.explode(index_parts=False)

# Exibir as primeiras linhas do dataframe
print(censo.head())

## JOIN

In [ ]:
# Filtrar apenas imóveis residenciais
IPTU_lote = IPTU[IPTU["residencial"] == 1]

# Realizar a junção com os lotes
IPTU_lote = IPTU_lote.merge(lotes, on=["setor", "quadra", "lote"], how="left")

# Converter para GeoDataFrame com o mesmo sistema de coordenadas dos lotes
IPTU_lote = gpd.GeoDataFrame(IPTU_lote, geometry=lotes.geometry, crs=lotes.crs)

# Exibir as primeiras linhas do dataframe
print(IPTU_lote.head())

In [ ]:


# Realizar a interseção espacial entre censo e IPTU_lote
IPTU_censo = gpd.overlay(censo, IPTU_lote, how='intersection')

# Converter para DataFrame tabular e renomear a coluna de geometria
IPTU_censo = IPTU_censo.rename(columns={"geometry": "geometria_intersec"})

# Recuperar geometrias do setor censitário
censo_geom = censo[["id_setor_censitario", "geometry"]].rename(columns={"geometry": "geometria_setor_censitario"})
IPTU_censo = IPTU_censo.merge(censo_geom, on="id_setor_censitario", how="left")

# Recuperar geometrias dos lotes
lotes_geom = IPTU_lote[["setor", "quadra", "lote", "geometry"]].rename(columns={"geometry": "geometria_lote"})
IPTU_censo = IPTU_censo.merge(lotes_geom, on=["setor", "quadra", "lote"], how="left")

# Calcular a porcentagem de interseção do lote dentro do setor
IPTU_censo["percent_intersec"] = IPTU_censo.apply(
    lambda row: row["geometria_intersec"].area / row["geometria_lote"].area if row["geometria_lote"] else 0, axis=1
)

# Exibir as primeiras linhas do dataframe
print(IPTU_censo.head())


## Cálculo do Erro:

In [ ]:
# Criar os dataframes com a coluna "base"
IPTU_bruta = IPTU[IPTU["residencial"] == 1].copy()
IPTU_bruta["base"] = "bruta"

IPTU_lote_filtered = IPTU_lote[~IPTU_lote.geometry.is_empty].copy()
IPTU_lote_filtered["base"] = "Lote"

# Função para agrupar e sumarizar unidades
def process_df(df):
    if "geometry" in df.columns:
        df = df.drop(columns=["geometry"])
    return df.groupby("base", as_index=False).agg(unidades=("unidades", "sum"))

# Aplicar a função aos dataframes
bruta_df = process_df(IPTU_bruta)
lote_df = process_df(IPTU_lote_filtered)

# Unir os resultados
result = pd.merge(bruta_df, lote_df, on="base", how="outer", suffixes=("_bruta", "_Lote"))

# Calcular diferenças
result["missing"] = result["unidades_bruta"] - result["unidades_Lote"]
result["missing_percent"] = (100 * result["missing"] / result["unidades_bruta"]).round(2).astype(str) + "%"

# Selecionar e renomear colunas
result = result.rename(columns={
    "unidades_Lote": "Critério de join",
    "missing": "Erros (unidades)",
    "missing_percent": "Percentual de erro"
})

# Exibir resultado
print(result)

## Gráficos
### Unidade vs Condomínio

In [ ]:

# Criar a coluna "Tipo" com base no número de unidades
IPTU["Tipo"] = IPTU["unidades"].apply(lambda x: "Unidade" if x == 1 else "Condomínio")

# Agrupar por "Tipo" e calcular os valores agregados
result = IPTU.groupby("Tipo", as_index=False).agg(
    Lotes=("Tipo", "count"),
    Unidades=("unidades", "sum")
)

# Exibir resultado
print(result)


In [ ]:

# Filtrar dados conforme as condições
filtered_data = IPTU_util[
    (IPTU_util["residencial"] == 1) &
    (IPTU_util["area_terreno"] < 10000) &
    (IPTU_util["area_construida"] < 5000) &
    (IPTU_util["valor_m2_terreno"] < 10000) &
    (IPTU_util["valor_m2_construcao"] < 10000)
]

# Selecionar e renomear colunas
filtered_data = filtered_data.rename(columns={
    "area_terreno": "Área do Terreno (m²)",
    "area_construida": "Área Construída (m²)",
    "valor_m2_terreno": "Preço do Terreno (R$/m²)",
    "valor_m2_construcao": "Preço da Construção (R$/m²)"
})

# Converter para formato longo para plotagem
melted_data = filtered_data.melt(var_name="Variável", value_name="Valor")

# Criar o gráfico
plt.figure(figsize=(12, 8))
g = sns.FacetGrid(melted_data, col="Variável", sharex=False, sharey=False, height=4)
g.map_dataframe(sns.histplot, x="Valor", bins=30, color="skyblue", edgecolor="black")
g.set_axis_labels("", "Frequência")
g.set_titles(col_template="{col_name}")
g.tight_layout()

plt.show()

In [ ]:

# Criar figura e eixo
fig, ax = plt.subplots(figsize=(10, 8))

# Definir colormap e escala logarítmica
norm = mcolors.LogNorm(vmin=1000, vmax=50000)
cmap = plt.get_cmap("plasma")

# Plotar mapa com escala logarítmica
IPTU_censo_valores.plot(
    column="valor_terreno",
    cmap=cmap,
    norm=norm,
    linewidth=0,
    legend=True,
    legend_kwds={
        "label": "Preço do Terreno (R$/m²)",
        "orientation": "vertical",
        "format": "%.0f"
    },
    ax=ax
)

# Ajustar título e remover eixos
ax.set_title("Distribuição do Preço Médio do Terreno (R$/m²)", fontsize=14)
ax.set_axis_off()

# Ajustar posição da legenda
leg = ax.get_legend()
leg.set_bbox_to_anchor((0.8, 0.3))

plt.show()
